In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import xarray as xr
import os
import glob

#### Prepare discharge data in mhm format

In [ ]:
# Read station data
stations_data_folder = r"W:/VUB/_main_research/mHM/model_validation_data/waterinfo_csv/waterinfo_timeseries/"
station_coords = pd.read_csv(r"W:/VUB/_main_research/mHM/model_validation_data/waterinfo_csv/waterinfo_timeseries/mhm_flow_stations_coordinates.csv")


for _,row in station_coords.iterrows():
    station_name = row.stat_name
    river_name = row.basin
    gauge_id = row.code

    # Extract the station data
    station_data = pd.read_csv(os.path.join(stations_data_folder, f"{station_name}_Q.csv"), index_col=0)

    # Convert index to datetime if it's not parsed correctly
    station_data.index = pd.to_datetime(station_data.index, dayfirst=True)

    #replace nan values with -9999
    station_data = station_data.fillna(-9999)


    output_file = f"W:/VUB/_main_research/mHM/model_validation_data/mhm_txt_gauges/{gauge_id}.txt"

    # Define the start and end dates
    # Extract the start and end dates from the data
    start_date = station_data.index.min()
    end_date = station_data.index.max()

    # Define the header information
    header = f"""{gauge_id} {station_name}/{river_name}  (Daily discharge)
    nodata   -9999
    n          1    measurements per day [1, 1440]
    start {start_date.year} {start_date.month:02d} {start_date.day:02d} 00 00    (YYYY MM DD HH MM)
    end   {end_date.year} {end_date.month:02d} {end_date.day:02d} 00 00    (YYYY MM DD HH MM)
    """

    # Prepare the measurement data with a single tab between each date component and the measurement
    # Prepare the measurement data with two spaces between each date component and the measurement
    data_lines = []
    for date, row in station_data.iterrows():
        # Format each row into the required format (YYYY  MM  DD  HH  MM  VALUE) with two spaces
        measurement = row['Q']
        date_str = f"{date.year}  {date.month:02d}  {date.day:02d}  {date.hour:02d}  {date.minute:02d}"
        data_lines.append(f"{date_str}   {measurement:.3f}")

    # Write to the file with two spaces between columns
    with open(output_file, 'w') as file:
        file.write(header)
        file.write("\n")
        for line in data_lines:
            file.write(line + "\n")

    print(f"Gauge data has been written to {output_file}", end="\r")